In [1]:
import numpy as np
import pickle
import cv2
import os
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Using TensorFlow backend.
C:\Users\ketan\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ketan\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ketan\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ketan\AppData\Roa

In [2]:
EPOCHS = 5
INIT_LR = 1e-3
BS = 32
default_image_size = tuple((128, 128))
image_size = 0
folder = "C:/Users/ketan/Desktop/hf/data/train"
width=128
height=128
depth=3

In [3]:
image_list = []

def load_images_from_folder(folder):
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            img = cv2.resize(img, default_image_size)
            img = img_to_array(img)
            image_list.append(img)
            
load_images_from_folder(folder)

In [4]:
np_image_list = np.array(image_list, dtype=np.float16) / 225.0

In [5]:
import pandas as pd
df = pd.read_csv("C:/Users/ketan/Desktop/hf/data/train.csv")

label_list = []
for i in range(len(df)):
    tags = df['category'][i]
    label_list.append(tags)

In [6]:
image_size = len(image_list)

In [7]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)
pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)
print(label_binarizer.classes_)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102]


In [8]:
print("[INFO] Spliting data to train, test")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

[INFO] Spliting data to train, test


In [9]:
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [10]:
model = Sequential()
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(n_classes))
model.add(Activation("softmax"))

In [11]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# distribution
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])
# train the network

In [12]:
print("[INFO] training network...")


history = model.fit_generator(
    aug.flow(x_train, y_train, batch_size=BS),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // BS,
    epochs=EPOCHS, verbose=1
    )

[INFO] training network...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5
463/463 [==============================] - 3546s 8s/step - loss: 0.0656 - accuracy: 0.9897 - val_loss: 0.0572 - val_accuracy: 0.9901
Epoch 2/5
463/463 [==============================] - 1647s 4s/step - loss: 0.0629 - accuracy: 0.9898 - val_loss: 0.0593 - val_accuracy: 0.9899
Epoch 3/5
463/463 [==============================] - 1696s 4s/step - loss: 0.0618 - accuracy: 0.9898 - val_loss: 0.0560 - val_accuracy: 0.9901
Epoch 4/5
463/463 [==============================] - 1740s 4s/step - loss: 0.0607 - accuracy: 0.9898 - val_loss: 0.0563 - val_accuracy: 0.9901
Epoch 5/5
463/463 [==============================] - 1770s 4s/step - loss: 0.0604 - accuracy: 0.9898 - val_loss: 0.0549 - val_accuracy: 0.9902


In [15]:
pickle.dump(model,open('cnn_model.pkl', 'wb'))

In [34]:
fill = "C:/Users/ketan/Desktop/hf/data/test"
data = []
lab = pickle.load(open('label_transform.pkl','rb'))
def load_images_from_folder(fill):
    for filename in os.listdir(fill):
        img = cv2.imread(os.path.join(fill,filename))
        if img is not None:
            ima = cv2.resize(img, default_image_size)
            ima = np.array(ima, dtype = np.float16) / 255.0
            ima = np.expand_dims(ima, axis=0)
            ans = model.predict(ima)
            ans = lab.inverse_transform(ans)
            data.append(ans)

In [35]:
load_images_from_folder(fill)

In [38]:
cat = np.ravel(data)

In [41]:
file = pd.read_csv("C:/Users/ketan/Desktop/hf/data/test.csv")
file['category'] = cat

In [42]:
file.head()

,image_id,category
0,18540,51
1,18541,77
2,18542,51
3,18543,51
4,18544,77


In [43]:
file.to_csv('C:/Users/ketan/Desktop/hf/data/final_submission.csv')